In [5]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
import pandas as pd

# Get all NBA teams
nba_teams = teams.get_teams()

# Find the Golden State Warriors
warriors = [team for team in nba_teams if team['full_name'] == 'Golden State Warriors'][0]
warriors_id = warriors['id']
print(f"Warriors Team ID: {warriors_id}")

season = '2023-24'  # Format: 'YYYY-YY'
season_type = 'Regular Season'  # Options: 'Regular Season', 'Playoffs', etc.

# Retrieve game logs
game_log = teamgamelog.TeamGameLog(team_id=warriors_id, season=season, season_type_all_star=season_type)
games_df = game_log.get_data_frames()[0]

# Extract the 'Game_ID' column
game_ids = games_df['Game_ID'].tolist()

# Display the game IDs
print("Warriors Game ID's", game_ids)


Warriors Team ID: 1610612744
Warriors Game ID's ['0022301198', '0022301182', '0022301169', '0022301155', '0022301142', '0022301097', '0022301113', '0022300589', '0022301085', '0022301062', '0022301049', '0022301044', '0022301031', '0022301016', '0022301001', '0022300988', '0022300973', '0022300952', '0022300936', '0022300921', '0022300907', '0022300899', '0022300872', '0022300860', '0022300852', '0022300834', '0022300824', '0022300811', '0022300802', '0022300576', '0022300789', '0022300769', '0022300757', '0022300733', '0022300727', '0022300717', '0022300699', '0022300693', '0022300673', '0022300650', '0022300632', '0022300626', '0022300560', '0022300545', '0022300536', '0022300525', '0022300504', '0022300490', '0022300478', '0022300463', '0022300444', '0022300426', '0022300402', '0022300399', '0022300386', '0022300362', '0022300348', '0022300339', '0022300325', '0022300308', '0022301222', '0022301214', '0022300280', '0022300273', '0022300060', '0022300051', '0022300236', '0022300224',

In [6]:
from nba_api.stats.endpoints import playbyplayv3
import time

def get_game_log(game_id):
    log = playbyplayv3.PlayByPlayV3(game_id=game_id)
    frames = log.get_data_frames()
    gamelog = frames[0]

    return gamelog

def file_exists(filename):
    try:
        with open(filename, 'r') as f:
            return True
    except FileNotFoundError:
        return False

csv = 'warriorsplaybyplay.csv'

if file_exists(csv):
    gamelog = pd.read_csv(csv)
else:
    gamelog = None
    for gameid in game_ids:
        time.sleep(1)  # Sleep for 1 second to avoid hitting the API too quickly
        log = get_game_log(gameid)
        # log = log[log['teamTricode'] == 'GSW']
        if gamelog is None:
            gamelog = log
        else:
            gamelog = pd.concat([gamelog, log])

    gamelog.to_csv(csv, index=False)

gamelog.loc[gamelog['teamTricode'] != 'GSW', 'teamTricode'] = 'OPP'
gamelog = gamelog[gamelog['actionType'] != 'period']
gamelog = gamelog[gamelog['actionType'] != 'Instant Replay']
gamelog = gamelog[gamelog['actionType'] != 'Substitution']
gamelog = gamelog[gamelog['actionType'] != '']


gamelog['state'] = gamelog['actionType'].fillna('') + "-" + gamelog['teamTricode']
gamelog['next_state'] = gamelog['state'].shift(-1)

transitions = gamelog.dropna(subset=['next_state'])
transition_counts = transitions.groupby(['state', 'next_state']).size().unstack(fill_value=0)
transition_matrix = transition_counts.div(transition_counts.sum(axis=1), axis=0)
transition_matrix.to_csv('transition_matrix.csv')
# transition_matrix = transition_matrix.to_numpy()

print(transition_matrix)


next_state           -GSW      -OPP  Ejection-GSW  Ejection-OPP  Foul-GSW  \
state                                                                       
-GSW             0.000000  0.000000      0.000000      0.000000  0.005269   
-OPP             0.000000  0.000000      0.000000      0.000000  0.089423   
Ejection-GSW     0.000000  0.000000      0.000000      0.000000  0.125000   
Ejection-OPP     0.000000  0.000000      0.500000      0.000000  0.250000   
Foul-GSW         0.000000  0.000000      0.002420      0.000605  0.037508   
Foul-OPP         0.000000  0.000000      0.000000      0.001970  0.003283   
Free Throw-GSW   0.000000  0.000000      0.000000      0.000000  0.055961   
Free Throw-OPP   0.000000  0.000000      0.000000      0.000000  0.011957   
Jump Ball-GSW    0.000000  0.000000      0.000000      0.000000  0.014286   
Jump Ball-OPP    0.000000  0.000000      0.000000      0.000000  0.065789   
Made Shot-GSW    0.000000  0.000000      0.000000      0.000000  0.098827   

In [7]:
import numpy as np

tm_np = transition_matrix.to_numpy()
eigvals, eigvectors = np.linalg.eig(transition_matrix)
print(transition_matrix.to_numpy().sum(axis=1))

print(eigvals)
print(transition_matrix.index)
eigvectors[0]

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.        +0.j         -0.13293918+0.38646601j -0.13293918-0.38646601j
 -0.37697902+0.j          0.26366233+0.00543114j  0.26366233-0.00543114j
  0.15387491+0.j         -0.05049426+0.14472237j -0.05049426-0.14472237j
 -0.132867  +0.j          0.01592983+0.10065503j  0.01592983-0.10065503j
 -0.09537018+0.j          0.04857713+0.03771604j  0.04857713-0.03771604j
 -0.01471469+0.05173114j -0.01471469-0.05173114j -0.05081157+0.j
 -0.01727587+0.j         -0.00643738+0.j          0.00299412+0.j        ]
Index(['-GSW', '-OPP', 'Ejection-GSW', 'Ejection-OPP', 'Foul-GSW', 'Foul-OPP',
       'Free Throw-GSW', 'Free Throw-OPP', 'Jump Ball-GSW', 'Jump Ball-OPP',
       'Made Shot-GSW', 'Made Shot-OPP', 'Missed Shot-GSW', 'Missed Shot-OPP',
       'Rebound-GSW', 'Rebound-OPP', 'Timeout-OPP', 'Turnover-GSW',
       'Turnover-OPP', 'Violation-GSW', 'Violation-OPP'],
      dtype='object', name='state')


array([-2.18217890e-01+0.j        , -1.57604626e-01-0.01570528j,
       -1.57604626e-01+0.01570528j, -1.97688611e-01+0.j        ,
        4.97541315e-02+0.00329384j,  4.97541315e-02-0.00329384j,
       -6.74478759e-02+0.j        ,  4.67915344e-02+0.06659455j,
        4.67915344e-02-0.06659455j,  1.03519594e-01+0.j        ,
       -1.89157823e-02-0.01495392j, -1.89157823e-02+0.01495392j,
       -1.06604062e-01+0.j        , -1.09235005e-04-0.00170084j,
       -1.09235005e-04+0.00170084j,  3.10783225e-03-0.00182433j,
        3.10783225e-03+0.00182433j,  3.52125431e-03+0.j        ,
       -6.44916178e-04+0.j        ,  2.42702945e-03+0.j        ,
        9.59735215e-04+0.j        ])

In [8]:
import numpy as np
import pandas as pd

# Assume transition_matrix is already defined as a DataFrame
P = transition_matrix.values

# Compute the left eigenvector associated with eigenvalue 1
eigvals, eigvecs = np.linalg.eig(P.T)

# Find the index of the eigenvalue closest to 1
idx = np.argmin(np.abs(eigvals - 1))
stationary = np.real(eigvecs[:, idx])

# Normalize to get a probability distribution
stationary = stationary / stationary.sum()

# Create a Series with labels
stationary_distribution = pd.Series(stationary, index=transition_matrix.index)

# Optional: sort the distribution by probability
stationary_distribution = stationary_distribution.sort_values(ascending=False)

print(stationary_distribution)


state
Rebound-OPP        0.134808
Missed Shot-OPP    0.110859
Missed Shot-GSW    0.110044
Rebound-GSW        0.107165
Made Shot-GSW      0.100224
Made Shot-OPP      0.096619
Free Throw-OPP     0.051488
Foul-GSW           0.046255
Free Throw-GSW     0.046002
Foul-OPP           0.042615
Turnover-GSW       0.031449
Turnover-OPP       0.031255
-OPP               0.029099
Timeout-OPP        0.027142
-GSW               0.026554
Violation-OPP      0.002658
Jump Ball-OPP      0.002127
Jump Ball-GSW      0.001931
Violation-GSW      0.001371
Ejection-GSW       0.000224
Ejection-OPP       0.000112
dtype: float64


Rebounds and missed shots dominate. 

In [9]:
import numpy as np
import pandas as pd

def simulate_game_flow(transition_matrix, steps=100, start_state=None):
    states = list(transition_matrix.index)
    P = transition_matrix.values

    # Create state index mapping
    state_to_index = {state: i for i, state in enumerate(states)}
    index_to_state = {i: state for state, i in state_to_index.items()}

    # If no start_state is provided, pick one randomly (weighted by stationary distribution)
    if start_state is None:
        stationary = np.real(np.linalg.eig(P.T)[1][:, np.argmax(np.isclose(np.linalg.eigvals(P.T), 1))])
        stationary = stationary / stationary.sum()
        current_index = np.random.choice(len(states), p=stationary)
    else:
        current_index = state_to_index[start_state]

    sequence = [index_to_state[current_index]]

    for _ in range(steps - 1):
        probs = P[current_index]
        next_index = np.random.choice(len(states), p=probs)
        sequence.append(index_to_state[next_index])
        current_index = next_index

    return sequence

# Example usage:
simulated_sequence = simulate_game_flow(transition_matrix, steps=50, start_state='Jump Ball-GSW')
for i, action in enumerate(simulated_sequence):
    print(f"{i + 1}. {action}")


1. Jump Ball-GSW
2. Made Shot-OPP
3. Made Shot-GSW
4. Foul-GSW
5. Made Shot-OPP
6. Missed Shot-GSW
7. Rebound-GSW
8. Made Shot-GSW
9. Foul-OPP
10. Timeout-OPP
11. Missed Shot-GSW
12. Rebound-GSW
13. Made Shot-GSW
14. Missed Shot-OPP
15. Rebound-OPP
16. Made Shot-OPP
17. Turnover-GSW
18. -OPP
19. Rebound-OPP
20. Timeout-OPP
21. Foul-GSW
22. Turnover-GSW
23. -OPP
24. Rebound-OPP
25. Missed Shot-OPP
26. Rebound-GSW
27. Turnover-GSW
28. -OPP
29. Rebound-OPP
30. Made Shot-OPP
31. Missed Shot-GSW
32. Rebound-GSW
33. Made Shot-GSW
34. Made Shot-OPP
35. Missed Shot-GSW
36. Rebound-OPP
37. Turnover-OPP
38. -GSW
39. Made Shot-GSW
40. Made Shot-OPP
41. Made Shot-GSW
42. Made Shot-OPP
43. Made Shot-GSW
44. Made Shot-OPP
45. Made Shot-GSW
46. Made Shot-OPP
47. Made Shot-GSW
48. Foul-GSW
49. Missed Shot-OPP
50. Rebound-GSW
